# Module 4 - Data Cleaning and Preparation

After getting an understanding of the dataset in the EDA phase, the data must then be cleaned and transformed for one or more of these next steps - statistical analysis, reporting (with data visualizations), or predictive modeling. You may end up needing different transformations of the same dataset for all of these purposes throughout the data project life cycle.

In this lesson, we will take the information gathered during the EDA to clean up the "messy" data (missing values, duplicates) and also restructure some columns of data into a more useable format, depending on the next step it will be used for.

In [ ]:
# import libraries
import pandas as pd
import numpy as np
import seaborn as sns

In [ ]:
# read in dataset
filepath = "datasets/gradedata_sn.csv"

df = pd.read_csv(filepath)
df.head(10)

In [ ]:
# check the data type of the columns
df.dtypes

## Replace values with null

Sometimes when data is being updated by a person/team that does not understand data storage best practices or the data was collected through surveys, missing or involuntary information is typicalled filled in as `NA`, `not applicable`, `unknown`, `-`, etc. When the data is then read into Python it interprets those filled values as strings, and if its mixed in a column that contains numeric data, `pandas` will assign the data type as an object.

We can replace the string values with an acutal missing information (blank cell) using the `np.nan` function, which creates a null value. To update the string values, use the syntax `df.loc[df[column] == string_val, column] = new_value`. This will select the rows where the column's value matches the string value and then within the same or a different column, assign the new value to those rows.

In [ ]:
# check the distinct categories in the 'gender' column
df['gender'].unique()

In [ ]:
# check where the "no answer" values are
df.loc[df['gender'] == 'no answer']

In [ ]:
# find the value "no answer" in the column and replace it w/ a null value
df.loc[df['gender'] == 'no answer', 'gender'] = np.nan

In [ ]:
# select the rows where the 'gender' value is missing (null)
# check if the values updated properly
df.loc[df['gender'].isnull()]

In [ ]:
# there are now 2 missing values in the 'gender' column
df.count()

## Change Column Data Type

Through additional exploration, you will find that the `age`, `exercise`, `hours`, and `grade` columns have `'no answer'` string values, rather than empty spaces for missing information. These columns have numeric data and even if the string values were replaced with null values, the data type of the column would still remain an object.

The `pd.to_numeric` will take in a column of data and convert the values to the numeric type (int or float) that it interprets. The argument `errors='coerce'` will convert any value that causes an error into a `NaN`. In this example, the strings values will cause an error because they cannot be converted to a number, and will be replaced with a null value.

### Object to Float type

In [ ]:
# convert the values in 'exercise' column to numbers
df['exercise'] = pd.to_numeric(df['exercise'], errors='coerce')

In [ ]:
# the data type of 'exercise' column is now float
df.dtypes

In [ ]:
# check that there are now missing values
df.count()

In [ ]:
# additionally convert 'hours' and 'grade' columns
df['hours'] = pd.to_numeric(df['hours'], errors='coerce')
df['grade'] = pd.to_numeric(df['grade'], errors='coerce')

df.dtypes

In [ ]:
# another check for missing values
df.count()

### Object to Int type

The `age` values in this dataset are formatted as float types (example: `19.0`) even though they are strings. Float types are typically treated as continuous quantitative values, but we may want to use `age` as a discrete quantitative data type, as we did during Modules 2 and 3. After converting the column to a numeric type (it will become a float) and replacing the string values with null, we will additionally use the `.astype()` function to truncate the decimal values and make the numbers as integers.

The `.astype()` function can be used to convert columns into int, float, or object data types.

In [ ]:
# convert data to numeric and replace string values w/ null
df['age'] = pd.to_numeric(df['age'], errors='coerce')

In [ ]:
df.dtypes

In [ ]:
# convert the column to an int type
# assign changes back into the 'age' column
df['age'] = df['age'].astype('Int64')

In [ ]:
# check column data types
df.dtypes

In [ ]:
# check for missing values
df.count()

## Rename columns

It is good practice to rename columns when column names in a dataset are too long, non-descriptive, or have hidden whitespaces (causing difficulties when selecting it).

The `.rename()` function uses a dictionary in the `columns=` argument, where the dictionary's key is the current column name, and the value is the new column name (similar to `.map`). Rather than directly assigning the changes back to the dataframe's variable name (in the style `df = df.whatever_function_made_changes()`), the `inplace=True` argument in the function will overwrite the dataframe's current information with the version that has the changes. This argument makes it easier to reuse the same variable name for the dataframe.

***Caution***: Be careful using `inplace=True`! Because it is a permanent change, if the changes are incorrect or unexpected, you will have to run the code cells starting from the beginning. Make a copy of the dataframe and experiment with the change first, and if everything is okay, then update the original dataframe using `inplace=True.

In [ ]:
# rename the 'fname' and 'lname' columns
df.rename(columns={'fname':'first_name', 'lname':'last_name'}, inplace=True)

In [ ]:
# check for the new column names
df.columns

## Remove Columns

In Module 2, you saw an example of creating a dataframe by selecting columns to keep. Another method of creating a dataframe without columns that are not needed (this is common when preparing the dataset for predictive modeling) is to drop the columns. 

The `.drop()` function takes in the column(s) name(s) and then argument `axis=1` directs it to find a column with that name(s) to drop. (***Note:*** A good method of remembering `axis=1` is that `1` is vertical, just like a column.)

### One Column

When dropping a single column, give the column name directly to the `.drop()` function as a string.

In [ ]:
# drop the 'address' column (permanent change using inplace=True)
df.drop('address', axis=1, inplace=True)

In [ ]:
# check the changes
df.head()

### Multiple Columns

When dropping multiple columns, give a list of column names to the `.drop()` function.

In [ ]:
# preview of changes (not saved)
df.drop(['first_name', 'last_name'], axis=1)

In [ ]:
df.head()

## Remove Duplicate Data

In Module 2, we identified rows of duplicate data using the `.duplicated()` function. If the duplicate rows are not necessary for the analysis, the `.drop_duplicates()` function will remove them from the dataset.

Similar to the `.duplicated()` function, the argument `subset=` can drop duplicates for a specific column(s). By default, the `.drop_duplicates()` function keeps the first instance of the duplicate (the first time that row of data was "seen") and drops any other duplicate rows found afterwards. Howver, the argument `keep='last'` will keep the last instance (last "seen") of the duplicated row, and remove all others beforehand (including the first instance).

In [ ]:
# create dataset with duplicates
names = ['Jessica','John','Bob','Jessica','Mary','John','Mel','Mel']
grades = [95,78,76,95,77,78,99,100]
GradeList = list(zip(names,grades))

df_dupe = pd.DataFrame(data = GradeList, columns=['Names', 'Grades'])
df_dupe

In [ ]:
# remove duplicate rows and assign changes to new dataframe
df_nodupe = df_dupe.drop_duplicates()
df_nodupe

In [ ]:
df_nodupe.loc[4]

In [ ]:
df_nodupe.iloc[3]

In [ ]:
df_nodupe.iloc[4]

In [ ]:
# remove 'Names' duplicates only and keep the last duplicate row
df_dupe.drop_duplicates(subset=['Names'], keep='last', inplace=True)
df_dupe

## Missing Data

Earlier in this lesson, we identified string values that were placeholders for missing information and then replaced them with actual null values. However, missing data does not add insight to our analysis and for predictive models, it cannot use rows with missing data. Missing data is handled in two ways - 1) fill in the missing information with a "guesstimate" or 2) exclude/eliminate rows with missing information.

### Replace Missing Values

Filling in a missing value with a "estimation" (called **imputation**) requires understanding about the dataset. Common values used to fill in missing data are averages (mean or median) or a zero value. Domain knowledge is useful when choosing an estimator value.

The `.fillna()` function replaces all missing values in a column with the estimator value given.

In [ ]:
# get the average age (of all non-null values)
# convert it to an "int" type (b/c of the column's data type)
age_mean = df['age'].mean()
age_mean = int(age_mean)

In [ ]:
# fill in missing ages with the mean age
df['age'].fillna(age_mean, inplace=True)

In [ ]:
# check that age does not have missing values
df.count()

### Remove Missing Values

When rows that have missing data are excluded/removed, it reduces the total number of rows in the dataset. 

***Caution:*** Some columns may have purposefully missing information, and removing those rows could result in a drastically smaller dataframe or an empty dataframe altogether.

In [ ]:
# total number of rows in dataframe
len(df)

### Exclude missing values

The best practice method of "removing" missing values is to exclude them, or in other words, select the rows that **do not** have missing information. Simialar to the `.isnull()` function, `.notnull()` assigns `True`/`False` boolean values depending on if the value is **not missing** (example: `True` that it is not missing).

In [ ]:
# select values that are not missing
# assign changes to new variable
df_nomissing = df.loc[df['gender'].notnull()]

In [ ]:
# number of rows in new dataframe
len(df_nomissing)

### Drop missing values

`.dropna` will remove rows from the dataframe on the condition that at least one value in the row is missing. To specify conditions for certain column(s) to check for missing values in order to drop the row, include the `subset=` argument.

In [ ]:
# drop rows that have missing values in the 'exercise' or 'hours' columns
df.dropna(subset=['exercise', 'hours'], inplace=True)

In [ ]:
# number of rows in the dataframe
len(df)

In [ ]:
df.dropna?

## Remove outliers

Outliers are values in the dataset that are uncommon or do not "behave" like other values. Outliers can skew our analysis or predictive model, if we are using the data for "normal" or typical patterns and trends.

Thr two methods we will use to exlcude outliers are the **Standard Deviation Method** and the **Interquartile Range Method**.

### Standard Deviation method

The Standard Deviation Method relies on the mean and standard deviation (range between mean and ~34% of the data) to determine the boundaries for outliers. Depending on the data, either 2 or 3 standard deviations is typically used for the boundary threshold.

In [ ]:
# calculate mean and standard deviation for 'grades' column
mean_grade = df['grade'].mean()
std_grade = df['grade'].std()

In [ ]:
# boundary value at +3std (3 standard deviations above the mean) and -3std (3 standard deviations below the mean)

upper_std = mean_grade + (3 * std_grade)
lower_std = mean_grade - (3 * std_grade)

In [ ]:
print(upper_std, lower_std)

In [ ]:
# select the rows between -3std and +3std (will filter out rows outside of those bounds)
df_std = df.loc[(df['grade'] >= lower_std) & (df['grade'] <= upper_std)]

In [ ]:
# compare the number of rows in the dataframe before and after removing outliers
print(len(df))
print(len(df_std))

### IQR method

The Interquartile Range Method relies on the median and IQR (interquartile range - innermost 50% of data) to determine the boundaries for outliers. The boundary values are **always** 1.5 times the IQR, above and below the median.

In [ ]:
# calculate the boundaries for 25th and 75th percentiles
Q1 = df['grade'].quantile(0.25)
Q3 = df['grade'].quantile(0.75)

# calculate IQR
IQR = Q3 - Q1

In [ ]:
# calculate upper and lower boundaries for outliers
upper_fence = Q3 + (1.5 * IQR)
lower_fence = Q1 - (1.5 * IQR)

In [ ]:
print(upper_fence, lower_fence)

In [ ]:
# select rows where the grade is between the lower and upper boundaries (not an outlier)
df_iqr = df.loc[(df['grade'] >= lower_fence) & (df['grade'] <= upper_fence)]

In [ ]:
# compare the number of rows in the dataframe before and after removing outliers
print(len(df))
print(len(df_iqr))

## Change Ordinal Categories to Discrete Numbers

Most machine learning algorithms cannot use string values for the columns used to predict an outcome. For qualitative categories that have order or rank, it is good practice to assign numerical order values to each category.

To reassign string values to numbers, we will use the `.map()` function. `.map` uses a dictionary, where the current value (value to reassign) is the key and the null value that it will be replaced with is the dictionary's value.

***Note:*** In a later lesson module, we will discuss **one-hot encoding**, which converts nominal (non-ordered) data into a boolean value representation.

In [ ]:
# create sample data
names = ['Bob','Jessica','Mary','John','Mel']
degrees = ['associates', 'phd', 'bachelors', 'associates', 'masters']
degree_list = list(zip(names, degrees))

df_degree = pd.DataFrame(data=degree_list, columns=['name', 'degree'])
df_degree

In [ ]:
# create dictionary with current values and numerical reassignment values
degree_val = {'associates':1, 'bachelors':2, 'masters':3, 'phd':4}

# give the dictionary to the .map function
# convert degree string categories to discrete numbers
df_degree['degree'] = df_degree['degree'].map(degree_val)

In [ ]:
# new version of 'degree' column
df_degree

## Create New Columns

We sometimes need to create new columns in the dataframe, that have information from a single column (transforming it in a different way) or is a combination of two or more columns' values. There are many methods to make new columns in a dataframe, and will vary depending on what kind of transformation/manipulation you need to do.

### Simple Math

In the `pandas` library, singular columns in a dataframe behave like an array. Using calculation methods from linrar algebra, arrays of the same shape can be combined together mathematically.

In [ ]:
# create sample dataset
names = ['Bob','Jessica','Mary','John','Mel']
bs = [1,1,0,0,1]
ms = [2,1,0,0,0]
phd = [0,1,0,0,0]

degrees_list = list(zip(names, bs, ms, phd))
column_names = ['name', 'bachelors', 'masters', 'phd']

df_degree = pd.DataFrame(data=degrees_list, columns=column_names)
df_degree

In [ ]:
# add the 'bachelors', 'masters', and 'phd' columns for total number of degrees
df_degree['num_degrees'] = df_degree['bachelors'] + df_degree['masters'] + df_degree['phd']

df_degree

### Represent continuous data as discrete (binning)

Continuous numerical data is good for calculating statistical values, such as averages (mean, median), standard deviations, minimum/maximum, etc. but are difficult to use when you want to analyze segments of similar groups within a range (such as people that have "middle income" within a particular salary range). We can create range groups from continuous data (called **binning**) to make discrete categories, represented with numbers or strings.

The `.apply()` function will operate a defined function on each value of a specified column in a dataframe. (***Note:*** If you find yourself wanting to loop through each row of a dataframe to do a set of tasks to a column, you most likely need to use the `.apply()` function. Looping on a dataframe is difficult and time/resource consuming.)

- use .apply (using def function on one column)
- lettergrade example

In [ ]:
# define a function that checks the grade value and assigns it a letter grade category
def lettergrade(grade_num):
    if grade_num < 60: # grade is 0-59
        grade ='F'
    elif grade_num < 70: # grade is 60-69
        grade = 'D'
    elif grade_num < 80: # grade is 70-79
        grade = 'C'
    elif grade_num < 90: # grade is 80-89
        grade = 'B'
    else: # grade is 90+
        grade = 'A'
        
    return grade

In [ ]:
# apply the 'lettergrade' function using the 'grade' column as the data source
# create a new column to store the new letter grade values
df['lettergrade'] = df['grade'].apply(lettergrade)

In [ ]:
# check the changes
df.head()

In [ ]:
# number of students per letter grade
pd.value_counts(df['lettergrade'])

In [ ]:
# average hours of study per letter grade
df.groupby('lettergrade')['hours'].mean()

### Use multiple columns

If you are using more than simple math to combine two or more columns' information, then you can define a function with the set of tasks and pass the columns into the function using the syntax `df[new_col] = func(col1, col2)`.

In [ ]:
# define a function that will combine two strings with a whitespace between them
# create a fullname for the student
def fullname(fname, lname):
    full_name = fname + " " + lname
    return full_name

In [ ]:
# use the 'first_name' and 'last_name' columns in the function
df['fullname'] = fullname(df['first_name'], df['last_name'])

In [ ]:
# check the changes
df.head()

### Make column headers into categories in a single column

Dataset structure is important for the kind of tasks that we want to do, not only for analysis in Python but also for data visualization. Often, categorical data within a single characteristic have their own separate columns and it makes it difficult to compare them, particularly for charts such as pie, bar, etc. 

The `pd.melt()` function takes columns that are categories, and reduces them to a single column. `id_vars=` takes a list of columns that **should not** be made into categories (are not included in the melt). `var_name=` assigns the new name of the column with categories, and `value_name` is the name of the columnthat stores the values that used to be in the categorical column.

In [ ]:
# create sample dataset
regions = ['East', 'South', 'Central', 'West']
q1_rev = [1500, 1200, 900, 8000]
q2_rev = [3000, 700, 1300, 6500]
q3_rev = [2700, 1000, 2600, 9400]
q4_rev = [4300, 1800, 3500, 9400]
quarter_rev = list(zip(regions, q1_rev, q2_rev, q3_rev, q4_rev))

column_names = ['region', 'Q1', 'Q2', 'Q3', 'Q4']

df_rev = pd.DataFrame(data=quarter_rev, columns=column_names)
df_rev

In [ ]:
# bar chart can only view one quarter at a time
# cannot compares all quarters again each other
sns.barplot(data=df_rev, x='region', y='Q1')

In [ ]:
# num_new_rows = num_old_rows * num_columns_melted

# column to NOT create into categories
do_not_melt = ['region']

# make a new column called 'quarter' to hold categories and 'revenue' will hold their values
melt_df = pd.melt(df_rev, id_vars=do_not_melt, var_name='quarter', value_name='revenue')
melt_df

In [ ]:
# now we can compare revenue for all the quarters across regions
sns.barplot(data=melt_df, x='quarter', y='revenue', hue='region')